This is a new file for doing the random forest algorith 

In [2]:
import numpy as np
from sklearn.externals import joblib
import pandas as pd

C:\Users\JungleBook\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [86]:
# Our lovely TFIDF Matrix 
tfidf_matrix = joblib.load('tfidf_matrix.pkl')

In [87]:
df = pd.read_csv('../Cleaning/opioid_tweets_final.csv').drop(columns = ["Unnamed: 0"])
y = df.is_bad
y = np.multiply(y, 1)

In [88]:
print(y)

0        0
1        0
2        0
3        0
4        0
        ..
42931    0
42932    0
42933    0
42934    0
42935    0
Name: is_bad, Length: 42936, dtype: int32


In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(tfidf_matrix ,y, test_size=0.2, random_state=0)

In [5]:
from sklearn.ensemble.forest import RandomForestClassifier #Be caresful with the  delcartion. 
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)


In [6]:
%timeit classifier.fit(X_train, y_train) # In order to proceed we need something to classify our text with, in the exapmle used they
#use the use y which contains 0 or 1. 
#It is possible to create script and flag the tweets we manully identified to have illicit drugs. 
#What else would we compare this against 

6min 41s ± 1.81 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
y_pred = classifier.predict(X_test)

ValueError: Number of features of the model must match the input. Model n_features is 20092 and input n_features is 20091 

In [11]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[8582    0]
 [   2    4]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8582
           1       1.00      0.67      0.80         6

    accuracy                           1.00      8588
   macro avg       1.00      0.83      0.90      8588
weighted avg       1.00      1.00      1.00      8588

0.9997671169073126


In [8]:
from sklearn.externals import joblib
joblib.dump(classifier, 'classifier.pkl')

['classifier.pkl']

In [31]:
classifier = joblib.load('classifier.pkl')

C:\Users\JungleBook\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\JungleBook\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.21.3 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\JungleBook\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.21.3 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Need to try resampling methods. Need to try with the true/false twitter url category. H-stack, concatinate matricies   

# URL as a estimator

In order to this, we must add the index column to "opioid_tweets_clean" then use  **np.hstack** to add "to_twitter" in the tfidf matrix. From there it will be used in the Random Forest and might make an impact. 

In [32]:
import numpy as np 
from sklearn.externals import joblib
import pandas as pd

In [94]:
to_twitter = df.twitter_url.fillna("True")#RIP, the True and False are saved as strings
to_twitter = np.multiply(to_twitter, 1)
for i in range(len(to_twitter)):
    if to_twitter[i] == "False":
        to_twitter[i] = 0
    else:
        to_twitter[i] = 1 


print(to_twitter)

0        1
1        1
2        1
3        0
4        0
        ..
42931    1
42932    1
42933    1
42934    1
42935    1
Name: twitter_url, Length: 42936, dtype: object


array([1, 1, 1, ..., 1, 1, 1], dtype=object)

In [69]:
tfidf_matrix.shape

(42936, 20091)

In [71]:
new_tfidf =  np.hstack((tfidf_matrix, to_twitter) )# when doing this we lose the row for some reason
#vstack does not work 

In [95]:
new_tfidf =  np.vstack((tfidf_matrix , to_twitter) )

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 42936

In [92]:
print(new_tfidf)

[<42936x20091 sparse matrix of type '<class 'numpy.float64'>'
	with 297166 stored elements in Compressed Sparse Row format>
 1 1 ... 1 1 1]


In [72]:
tfidf_matrix.shape

(42936, 20091)

In [73]:
new_tfidf.shape # Since to_twitter is will add more columns than there already was... it gets rid of them 
#The hstack does this... 

(42937,)

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(new_tfidf ,y, test_size=0.2, random_state=0)


ValueError: Found input variables with inconsistent numbers of samples: [42937, 42936]

In [46]:
from sklearn.ensemble.forest import RandomForestClassifier #Be caresful with the  delcartion. 
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)

C:\Users\JungleBook\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [48]:
%timeit classifier.fit(X_train, y_train) # In order to proceed we need something to classify our text with, in the exapmle used they
#use the use y which contains 0 or 1. 
#It is possible to create script and flag the tweets we manully identified to have illicit drugs. 
#What else would we compare this against 

ValueError: setting an array element with a sequence.

In [27]:
y_pred = classifier.predict(X_test)

In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[8588]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8588

    accuracy                           1.00      8588
   macro avg       1.00      1.00      1.00      8588
weighted avg       1.00      1.00      1.00      8588

1.0


# Oversampling 